In [0]:
from delta.tables import DeltaTable
from pyspark.sql import functions as F

DELTA_PATH = "dbfs:/Volumes/workspace/ecommerce/ecommerce_data/delta/events"


In [0]:
# Load Delta table
deltaTable = DeltaTable.forPath(spark, DELTA_PATH)


In [0]:
current = spark.read.format("delta").load(DELTA_PATH)
updates_existing = (
    current.orderBy(F.rand()).limit(5000)
    .withColumn("price", (F.col("price") * 1.05))
)
inserts_new = (
    current.orderBy(F.rand()).limit(5000)
    .withColumn("event_time", F.col("event_time") + F.expr("INTERVAL 1 DAY"))
)

updates = updates_existing.unionByName(inserts_new)

deltaTable.alias("t").merge(
    updates.alias("s"),
    """
    t.user_session = s.user_session
    AND t.event_time = s.event_time
    AND t.event_type = s.event_type
    AND t.product_id = s.product_id
    """
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
inserts_new = (
    current.orderBy(F.rand()).limit(5000)
    .withColumn("event_time", F.col("event_time") + F.expr("INTERVAL 1 DAY"))
)

updates = updates_existing.unionByName(inserts_new)

deltaTable.alias("t").merge(
    updates.alias("s"),
    """
    t.user_session = s.user_session
    AND t.event_time = s.event_time
    AND t.event_type = s.event_type
    AND t.product_id = s.product_id
    """
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
v0 = spark.read.format("delta").option("versionAsOf", 0).load(DELTA_PATH)
old_snapshot = spark.read.format("delta") \
    .option("timestampAsOf", "2026-01-13T12:00:00.000+0000") \
    .load(DELTA_PATH)

In [0]:
spark.sql("OPTIMIZE events_table ZORDER BY (event_type, user_id)")
spark.sql("VACUUM events_table RETAIN 168 HOURS")

DataFrame[path: string]

In [0]:
spark.read.format("delta").load(DELTA_PATH).limit(20).show(truncate=False)


+-------------------+----------+----------+-------------------+-----------------------------------+-------+------+---------+------------------------------------+
|event_time         |event_type|product_id|category_id        |category_code                      |brand  |price |user_id  |user_session                        |
+-------------------+----------+----------+-------------------+-----------------------------------+-------+------+---------+------------------------------------+
|2019-10-16 10:49:08|view      |26201166  |2053013563693335403|NULL                               |lucente|76.45 |558367663|82d892bf-14c5-4c14-b86c-0b6f7acc383e|
|2019-10-16 10:49:08|view      |5800311   |2053013553945772349|electronics.audio.subwoofer        |jvc    |23.17 |525247825|f6fb2784-3fec-4172-bc25-3a49b4f77d62|
|2019-10-16 10:49:08|view      |3900339   |2053013552326770905|appliances.environment.water_heater|aqua   |30.86 |552776988|2cdf22dd-88f1-420b-bc01-f0e3cd0411da|
|2019-10-16 10:49:08|view   